## Generate images for evaluation

In [ ]:
import os,json
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import sys
utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import torch
from diffusers import DiffusionPipeline, UNet2DConditionModel
from transformers import CLIPTextModel
from utils.dataset_info import get_subjects_prompts_info


# Single Subject Generation
single_subject = []                            # "backpack"
# Single Prompt Generation
single_prompt = []                            # e.g. ["a {0} {1} near the pool"]

num_generation = 4


output_path = "../../outputs/subjects/dreambooth"
embedding_folds = os.listdir("../../logs/subjects/dreambooth/")
dataset_info_path = "../../pcs_dataset/info.json"
os.makedirs(output_path, exist_ok=True)

prompts_info = get_subjects_prompts_info(dataset_info_path)

if len(single_subject):
    subjects = single_subject

for subject in subjects:

    if subject not in os.listdir(output_path):

        print(f"***** Subject: {subject} *****")

        # Load the pipeline with the same arguments (model, revision) that were used for training
        unet = UNet2DConditionModel.from_pretrained(f"../../logs/subjects/dreambooth/{subject}/checkpoint-500/unet")
        # if you have trained with `--args.train_text_encoder` make sure to also load the text encoder
        # text_encoder = CLIPTextModel.from_pretrained("/sddata/dreambooth/daruma-v2-1/checkpoint-100/text_encoder")
        pipeline = DiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", unet=unet, dtype=torch.float16, use_safetensors=True)
        pipeline.to("cuda")

        if len(single_prompt):
            prompts = single_prompt
        else:
            prompts = prompts_info[subject]["prompts"]

        for prompt in prompts:
            prompt = prompt.replace("{0}", "sks")
            prompt = prompt.replace("{1}", f"{prompts_info[subject]['class']}")
            print(f"**Prompt**: {prompt}")

            image = pipeline(prompt, num_inference_steps=50, num_images_per_prompt=4).images

            save_path = os.path.join(output_path, subject, f"{prompt}")
            os.makedirs(save_path, exist_ok=True)

            for img_idx, img in enumerate(image):
                img.save(os.path.join(save_path, f"{img_idx:04d}.jpg"))
        
        print(f"Finished dreambooth in subject: {subject}!")

## Evaluation

Calculate img to img similarity and text to img similarity by CLIP Evaluator

In [3]:
import os, sys, json

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

utils_path = os.path.abspath(os.path.join('../..'))
sys.path.append(utils_path)

import numpy as np
from utils.clip_eval import CLIPEvaluator, evaluate_i2i, evaluate_t2i
import re
from utils.dataset_info import get_subjects_prompts_info

In [4]:
def extract_id(folder_name):
    match = re.search(r'\d+', folder_name)
    if match:
        return int(match.group())
    return None

def convert_to_native(data):
    if isinstance(data, np.ndarray):
        return data.tolist()
    elif isinstance(data, np.generic):
        return data.item()
    elif isinstance(data, dict):
        return {key: convert_to_native(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [convert_to_native(item) for item in data]
    else:
        return data

In [ ]:
outputs_path = "../../outputs/subjects/dreambooth"
eval_res_path = "../../eval_results/subjects/dreambooth"
dataset_path = "../../pcs_dataset/subjects"
dataset_info_path = "../../pcs_dataset/info.json"

prompts_info, prompts = get_subjects_prompts_info(dataset_info_path)

subjects_list = os.listdir(outputs_path)

evaluation_res = dict()
for subject in subjects_list:
    print("process subject: ", subject)

    res_for_each_subject =dict()

    for sub_dir in os.listdir(os.path.join(outputs_path, subject)):
        print("prompt: ", sub_dir)

        prompt = prompts[prompts_info[subject]["prompt_type"]][extract_id(sub_dir)]
        prompt = prompt.replace("{0} {1}", prompts_info[subject]["class"])
        res_for_each_prompt =dict()

        for generate_img_name in os.listdir(os.path.join(outputs_path, subject, sub_dir)):
            generate_img_path = os.path.join(outputs_path, subject, sub_dir, generate_img_name)
            res_for_each_prompt[generate_img_name] = [evaluate_i2i(generate_img_path, os.path.join(dataset_path, subject)), evaluate_t2i(generate_img_path, prompt)]
        
        res_for_each_subject[prompt] = res_for_each_prompt

        print(res_for_each_prompt)

    evaluation_res[subject] = res_for_each_subject

os.makedirs(eval_res_path, exist_ok=True)

with open(os.path.join(eval_res_path, "evaluation_results.json"), "w") as json_file:
    json.dump(convert_to_native(evaluation_res), json_file, indent=4)